# Demo - Amazon Rekognition, Amazon Textract e Amazon Comprehend

Vamos utilizar algumas das APIs de alguns serviços de inteligência artificial para extair informações de imagens e textos

In [ ]:
import boto3 # aws python sdk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import IPython.display as disp
from IPython.display import Markdown

In [ ]:
image_dir = '../sample-data/gov/images/'
text_dir = '../sample-data/gov/text/'

## Funções auxiliares

In [ ]:
def load_image(filename):
    with open(filename, "rb") as imageFile:
      f = imageFile.read()
      return bytearray(f)

In [ ]:
def convert_to_dataframe(labels):
    data = {'Label':[], 'Confidence':[]}
    for label in labels:
        data['Label'].append(label['Name'])
        data['Confidence'].append(label['Confidence'])
    return pd.DataFrame(data)[['Label', 'Confidence']]

In [ ]:
def create_bounding_box(bbox, size):
    if len(bbox) != 4:
        return None
    return [ 
        bbox['Left']*size[0], bbox['Top']*size[1],
        (bbox['Left']*size[0])+bbox['Width']*size[0], 
        (bbox['Top']*size[1])+bbox['Height']*size[1]
    ]

In [ ]:
def draw_bounding_box(filename, bbox, color, size):
    img = Image.open(filename)
    draw = ImageDraw.Draw(img)
    draw.line([(bbox[0], bbox[1]), (bbox[2], bbox[1])], fill=color, width=4)
    draw.line([(bbox[2], bbox[1]), (bbox[2], bbox[3])], fill=color, width=4)
    draw.line([(bbox[2], bbox[3]), (bbox[0], bbox[3])], fill=color, width=4)
    draw.line([(bbox[0], bbox[1]), (bbox[0], bbox[3])], fill=color, width=4)
    del draw
    plt.figure(figsize = (20,size))
    plt.imshow(img)

In [ ]:
def print_lines_confidence(lines, lenght):
    try:
        for l in range(1, lenght):
            print(lines[l]['Text'] + '({})'.format(lines[l]['Confidence']))
    except Exception as e:
        print("Too long")

In [ ]:
def delete_collection(name):
    try:
        response = rekognition.delete_collection(
            CollectionId=name
        )
    except Exception as e:
        print("Collection not found")

In [ ]:
def delete_classifier():
    pass

In [ ]:
def delete_entity():
    pass

## Amazon Rekognition

O Amazon Rekognition facilita a adição de análises de imagens e vídeos aos seus aplicativos. Basta fornecer uma imagem ou um vídeo à API do Amazon Rekognition, e o serviço poderá identificar objetos, pessoas, texto, cenas e atividades. 

Documentação: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html

_fontes das imagens utilizadas:_ 
* http://agenciabrasil.ebc.com.br/
* http://g1.globo.com/
* https://www.jornalterceiravia.com.br/

In [ ]:
# api client
rekognition = boto3.client('rekognition')

### Detecção de texto

In [ ]:
disp.Image(image_dir + 'carro.jpg', width=400)

In [ ]:
response = rekognition.detect_text(
    Image={'Bytes': load_image(image_dir + 'carro.jpg')}
)

for i in response['TextDetections']:
    if i['Confidence'] >= 95:
        print( '%s - Confidence[%f]' % (i['DetectedText'], i['Confidence']) )

### Detecção de objetos

In [ ]:
disp.Image(image_dir + 'agenciabrasil-4.jpg', width=400)

In [ ]:
response = rekognition.detect_labels(
    Image={'Bytes': load_image(image_dir + 'agenciabrasil-4.jpg')},
    MaxLabels=5,
    MinConfidence=70
)

convert_to_dataframe(response['Labels'])

### Moderação

In [ ]:
disp.Image(image_dir + 'arma.jpg', width=300)

In [ ]:
response = rekognition.detect_moderation_labels(
    Image={'Bytes': load_image(image_dir + 'arma.jpg')},
    MinConfidence=70
)

convert_to_dataframe(response['ModerationLabels'])

### Reconhecimento de pessoas públicas

In [ ]:
disp.Image(image_dir + 'agenciabrasil-2.jpg', width=300)

In [ ]:
response = rekognition.recognize_celebrities(
    Image={'Bytes': load_image(image_dir + 'agenciabrasil-2.jpg')}
)

img = Image.open(image_dir + 'agenciabrasil-2.jpg')
bbox = create_bounding_box(response['CelebrityFaces'][0]['Face']['BoundingBox'], img.size )
confidence = response['CelebrityFaces'][0]['Face']['Confidence']
name = response['CelebrityFaces'][0]['Name']

print(name, confidence)
draw_bounding_box(image_dir + 'agenciabrasil-2.jpg', bbox, 'red', 4)

### Análise facial

In [ ]:
disp.Image(image_dir + 'agenciabrasil-5.jpg', width=400)

In [ ]:
response = rekognition.detect_faces(
    Image={'Bytes': load_image(image_dir + 'agenciabrasil-5.jpg')},
    Attributes=['ALL']
)

age = response['FaceDetails'][0]['AgeRange']['High']
gender = response['FaceDetails'][0]['Gender']['Value']

print(age, gender)

age = response['FaceDetails'][1]['AgeRange']['High']
gender = response['FaceDetails'][1]['Gender']['Value']

print(age, gender)

### Comparação facial

In [ ]:
disp.Image(image_dir + 'jk-1.jpg', width=300)

In [ ]:
disp.Image(image_dir + 'jk-2.jpg', width=300)

In [ ]:
response = rekognition.compare_faces(
    SourceImage={'Bytes': load_image(image_dir + 'jk-1.jpg')},
    TargetImage={'Bytes': load_image(image_dir + 'jk-2.jpg')}
)

imgA = Image.open(image_dir + 'jk-1.jpg')
imgB = Image.open(image_dir + 'jk-2.jpg')

similarity = response['FaceMatches'][0]['Similarity']
bboxA = create_bounding_box( response['SourceImageFace']['BoundingBox'], imgA.size )
bboxB = create_bounding_box( response['FaceMatches'][0]['Face']['BoundingBox'], imgB.size )

In [ ]:
print( 'Similarity: {}'.format( similarity ) )
if bboxA: draw_bounding_box(image_dir + 'jk-1.jpg', bboxA, 'red', 5)

In [ ]:
if bboxB: draw_bounding_box(image_dir + 'jk-2.jpg', bboxB, 'white', 5)

### Busca por faces

O Rekognition possui uma feature para que possamos criar coleções de faces, efetuando uma busca otimizada ao receber uma nova face

In [ ]:
disp.Image(image_dir + 'tiririca-1.jpg', width='400')

In [ ]:
try:
    delete_collection("Deputados")
    response = rekognition.create_collection(
        CollectionId='Deputados'
    )
    faces = {
        'Tiririca': image_dir + 'tiririca-1.jpg'
    }
    for ext_id, image_name in faces.items():
        response = rekognition.index_faces(
            CollectionId='Deputados',
            Image={'Bytes': load_image(image_name)},
            ExternalImageId=ext_id,
        )
        if len(response['FaceRecords']) > 0:
            for i in response['FaceRecords']:
                print( ext_id, i['Face']['FaceId'])
except Exception as e:
    print(e)

Agora iremos enviar uma imagem diferente da utilizada na coleção e buscar por faces conhecidas pela coleção

In [ ]:
disp.Image(image_dir + 'tiririca-2.jpg', width='400')

In [ ]:
faces = rekognition.search_faces_by_image(
    CollectionId='Deputados',
    Image={ 'Bytes': load_image( image_dir + 'tiririca-2.jpg') },
    MaxFaces=10
)['FaceMatches']

print('# Mached faces in the collection: {}'.format( len(faces)))
print('FaceId: {} - Similarity: {}'.format(faces[0]['Face']['ExternalImageId'], faces[0]['Similarity']))

## Amazon Textract

O Amazon Textract é um serviço que extrai automaticamente texto e dados de documentos digitalizados. O Amazon Textract vai além do simples OCR (Reconhecimento óptico de caracteres) para também identificar o conteúdo de campos em formulários e informações armazenadas em tabelas.

Documentação: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/textract.html

In [ ]:
# api client
textract = boto3.client('textract')

### OCR

Síncrono com suporte a documentos JPEG e PNG. Assíncrono com suporte a dcoumentos JPEG, PNG e PDF. Neste exemplo estamos utilizando o método síncrono.

In [ ]:
disp.Image(text_dir + 'constituicao-pg220.jpg', width=600)

In [ ]:
response = textract.detect_document_text(
    Document={
        'Bytes': load_image(text_dir + 'constituicao-pg220.jpg')
    }
)

text = response['Blocks'][1]['Text'] # storing text to use with Comprehend

print_lines_confidence(response['Blocks'], 5)

In [ ]:
# bbox on the first line
img = Image.open(text_dir + 'constituicao-pg220.jpg')
bbox = create_bounding_box( response['Blocks'][1]['Geometry']['BoundingBox'], img.size )
draw_bounding_box(text_dir + 'constituicao-pg220.jpg', bbox, 'red', 12)

## Amazon Comprehend

O Amazon Comprehend é um serviço de Natural Language Processing (NLP – Processamento de linguagem natural) que usa Machine Learning para encontrar insights e relações dentro de documentos em texto. Nenhuma experiência de Machine Learning necessária.

Também é possível trabalhar de forma síncrona e assíncrona para análise de texto. Neste exemplo estamos utilizando o método síncrono.

Documentação: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend.html

In [ ]:
# api client
comprehend = boto3.client('comprehend')

### Detecção de linguagem

In [ ]:
print(text)

response = comprehend.detect_dominant_language(Text = text)

print(response['Languages'][0]['LanguageCode'] + ' - Confidence: {}'.format(response['Languages'][0]['Score']))

### Detecção de entidades

In [ ]:
response = comprehend.detect_entities(
    Text = text,
    LanguageCode = 'pt'
)

for e in range(0, len(response['Entities'])):
    print(response['Entities'][e]['Type'] + ' - ' + response['Entities'][e]['Text'] + ' - ' + str(response['Entities'][e]['Score']))

### Detecção de palavras chaves

In [ ]:
response = comprehend.detect_key_phrases(
    Text = text,
    LanguageCode = 'pt'
)

print(response['KeyPhrases'][0]['Text'])
print(response['KeyPhrases'][1]['Text'])

### Detecção de sentimento

In [ ]:
response = comprehend.detect_sentiment(
    Text = text,
    LanguageCode = 'pt'
)

print(response['Sentiment'] + ' - Accuracy: {}'.format(response['SentimentScore']))

### Análise Morfológica

In [ ]:
response = comprehend.detect_syntax(
    Text = text,
    LanguageCode = 'pt'
)

for t in range(0, len(response['SyntaxTokens'])):
    print(response['SyntaxTokens'][t]['Text'] + ' - ' + response['SyntaxTokens'][t]['PartOfSpeech']['Tag'] + ' - ' + str(response['SyntaxTokens'][t]['PartOfSpeech']['Score']))

## Limpando os recursos criados

In [ ]:
# Rekognition collection
delete_collection('Deputados')